In [9]:
import inspect
import time
from statistics import mean, stdev
from CybORG import CybORG
from CybORG.Agents import B_lineAgent, SleepAgent, GreenAgent
from CybORG.Agents.SimpleAgents.BaseAgent import BaseAgent
from CybORG.Agents.SimpleAgents.BlueReactAgent import BlueReactRemoveAgent
from CybORG.Agents.SimpleAgents.Meander import RedMeanderAgent
from CybORG.Agents.Wrappers.EnumActionWrapper import EnumActionWrapper
from CybORG.Agents.Wrappers.FixedFlatWrapper import FixedFlatWrapper
from CybORG.Agents.Wrappers.OpenAIGymWrapper import OpenAIGymWrapper
from CybORG.Agents.Wrappers.ReduceActionSpaceWrapper import ReduceActionSpaceWrapper
from CybORG.Agents.Wrappers import ChallengeWrapper
import os

from ray.tune.registry import register_env
from CybORG.Agents.Wrappers.rllib_wrapper import RLlibWrapper
import warnings
import numpy as np
from ray import air, tune
warnings.filterwarnings('ignore')

In [10]:

from ray.rllib.utils.framework import try_import_tf
tf1, tf, tfv = try_import_tf()
tf.test.gpu_device_name()

'/device:GPU:0'

In [11]:
!export PATH=/home/adamprice/miniconda3/envs/u75a/lib${PATH:+:${PATH}}


In [12]:
!export LD_LIBRARY_PATH=/home/adamprice/miniconda3/envs/u75a/lib:${LD_LIBRARY_PATH:+:${LD_LIBRARY_PATH}}

In [13]:
import tensorflow
tensorflow.test.gpu_device_name()


'/device:GPU:0'

In [14]:
def env_creator(env_config: dict):
    path = str(inspect.getfile(CybORG))
    path = path[:-10] + '/Shared/Scenarios/Scenario2_No_Decoy.yaml'
    #RedMeanderAgent B_lineAgent
    agents = {"Red": RedMeanderAgent, "Green": GreenAgent}
    cyborg = CybORG(scenario_file=path, environment='sim', agents=agents)
    env = RLlibWrapper(env=cyborg, agent_name="Blue", max_steps=100)
    return env

def print_results(results_dict):
    train_iter = results_dict["training_iteration"]
    r_mean = results_dict["episode_reward_mean"]
    r_max = results_dict["episode_reward_max"]
    r_min = results_dict["episode_reward_min"]
    print(f"{train_iter:4d} \tr_mean: {r_mean:.1f} \tr_max: {r_max:.1f} \tr_min: {r_min: .1f}")

register_env(name="CybORG", env_creator=env_creator)

In [15]:

tune.Tuner(
        "PPO",
        run_config=air.RunConfig(
            stop={"timesteps_total": 1e6},
            local_dir='results/PPO', name="ppo_tune_nodecoy_meander",
            checkpoint_config=air.CheckpointConfig(
                checkpoint_frequency=500, 
            ),
        ),
        param_space={
            # CC3 specific.
            "env": "CybORG",
            # General
            "num_gpus": 1,
            "num_workers": 20,
            "horizon": 100,
            "num_envs_per_worker": 1,
            #"exploration_config": tune.grid_search([{"type": "RE3",
            #    "embeds_dim": 128,
            #    "beta_schedule": "constant",
            #    "sub_exploration": {
            #        "type": "StochasticSampling",
            #},}, 
            "exploration_config": {"type": "StochasticSampling"},
            #algo params
            "train_batch_size": tune.grid_search([1000, 2000, 4000]),
            "lr": tune.grid_search([0.0005, 0.0001, 0.00005]),
            "gamma": 0.9,
            "framework": 'tf',
            "model": {
                    "fcnet_hiddens": [256, 256],
                    "fcnet_activation": "tanh",

                },
        },
    ).fit()

2023-03-21 18:28:04,680	INFO algorithm_config.py:2503 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2023-03-21 18:28:04,682	INFO algorithm_config.py:2503 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2023-03-21 18:28:04,686	INFO algorithm_config.py:2503 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2023-03-21 18:28:04,687	INFO algorithm_config.py:2503 -- Your framework setting is

(pid=38143) /home/adamprice/.local/lib/python3.8/site-packages/h5py/__init__.py:46: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
(pid=38143)   from ._conv import register_converters as _register_converters
(pid=38143) /home/adamprice/.local/lib/python3.8/site-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
(pid=38143)   from numpy.dual import register_func
(pid=38143) /home/adamprice/.local/lib/python3.8/site-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
(pid=38143)   supported_dtypes = [np.typeDict[x] for x in supported_dtypes]
(pid=38143) /home/adamprice/.local/lib/python3.8/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
(pid=38143)   import imp
(pid=38143

In [16]:
!pip show ray

Name: ray
Version: 2.2.0
Summary: Ray provides a simple, universal API for building distributed applications.
Home-page: https://github.com/ray-project/ray
Author: Ray Team
Author-email: ray-dev@googlegroups.com
License: Apache 2.0
Location: /home/adamprice/miniconda3/envs/u75a/lib/python3.8/site-packages
Requires: aiosignal, attrs, click, filelock, frozenlist, grpcio, jsonschema, msgpack, numpy, protobuf, pyyaml, requests, virtualenv
Required-by: CybORG
